In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
plt.style.use("fivethirtyeight")

# create element tree object
tree = ET.parse('apple_health_export/Export.xml') 
# for every health record, extract the attributes
root = tree.getroot()
record_list = [x.attrib for x in root.iter('Record')]

In [3]:
record_data = pd.DataFrame(record_list)

# proper type to dates
for col in ['creationDate', 'startDate', 'endDate']:
    record_data[col] = pd.to_datetime(record_data[col])

# value is numeric, NaN if fails
record_data['value'] = pd.to_numeric(record_data['value'], errors='coerce')

# some records do not measure anything, just count occurences
# filling with 1.0 (= one time) makes it easier to aggregate
record_data['value'] = record_data['value'].fillna(1.0)

# shorter observation names
record_data['type'] = record_data['type'].str.replace('HKQuantityTypeIdentifier', '')
record_data['type'] = record_data['type'].str.replace('HKCategoryTypeIdentifier', '')
record_data.tail()

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
296136,AppleWalkingSteadiness,Bhagavan,2890.16.16,%,2024-07-20 17:34:23-07:00,2024-07-13 17:00:00-07:00,2024-07-20 17:00:00-07:00,0.949619,"<<HKDevice: 0x302070140>, name:iPhone, manufac..."
296137,AppleWalkingSteadiness,Bhagavan,2890.16.16,%,2024-07-27 23:06:19-07:00,2024-07-20 17:00:00-07:00,2024-07-27 17:00:00-07:00,0.936161,"<<HKDevice: 0x302070140>, name:iPhone, manufac..."
296138,AppleWalkingSteadiness,Bhagavan,2890.16.16,%,2024-08-03 23:12:36-07:00,2024-07-27 17:00:00-07:00,2024-08-03 17:00:00-07:00,0.971380,"<<HKDevice: 0x302070140>, name:iPhone, manufac..."
296139,AppleWalkingSteadiness,Bhagavan,2890.16.16,%,2024-08-10 19:30:42-07:00,2024-08-03 17:00:00-07:00,2024-08-10 17:00:00-07:00,0.939884,"<<HKDevice: 0x302070140>, name:iPhone, manufac..."
296140,AppleWalkingSteadiness,Bhagavan,2890.16.16,%,2024-08-17 21:52:53-07:00,2024-08-10 17:00:00-07:00,2024-08-17 17:00:00-07:00,0.974364,"<<HKDevice: 0x302070140>, name:iPhone, manufac..."


In [7]:
workout_list = [x.attrib for x in root.iter('apple_health_export/Export_cda.xml')]

# create DataFrame
workout_data = pd.DataFrame(workout_list)
workout_data['workoutActivityType'] = workout_data['workoutActivityType'].str.replace('HKWorkoutActivityType', '')
workout_data = workout_data.rename({"workoutActivityType": "Type"}, axis=1)

# proper type to dates
for col in ['creationDate', 'startDate', 'endDate']:
    workout_data[col] = pd.to_datetime(workout_data[col])

# convert string to numeric   
workout_data['duration'] = pd.to_numeric(workout_data['duration'])
workout_data['totalEnergyBurned'] = pd.to_numeric(workout_data['totalEnergyBurned'])
workout_data['totalDistance'] = pd.to_numeric(workout_data['totalDistance'])
workout_data.tail()

KeyError: 'workoutActivityType'

In [10]:
def get_heartrate_for_workout(heartrate, workout):
    def get_heartrate_for_date(hr, start, end):
        hr = hr[hr["startDate"] >= start]
        hr = hr[hr["endDate"] <= end]
        return hr
    return get_heartrate_for_date(heartrate, workout["startDate"].item(), workout["endDate"].item())

heartrate_data = record_data[record_data["type"] == "HeartRate"]

# Extract heartrate statistics for certain workout
last_workout = workouts.iloc[[-1]]
heartrate_workout = get_heartrate_for_workout(heartrate_data, last_workout)
minh = heartrate_workout["value"].min()
maxh = heartrate_workout["value"].max()
meanh = heartrate_workout["value"].mean()
print(last_workout.Type.item(), minh, maxh, meanh)
# HighIntensityIntervalTraining 74.0 176.0 151.2590909090909

NameError: name 'workouts' is not defined